In [1]:
from fyers_apiv3.FyersWebsocket import data_ws
from datetime import datetime

import sqlite3 as sql
from utils import login

import pandas as pd

# Login Process


In [2]:
access_token, fyers = login()
fyers.get_profile()

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=KKSQ1PYF0H-100&redirect_uri=https%3A%2F%2Fsabyabhoi.xyz%2F&response_type=code&state=sample


{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'DS05962',
  'name': 'SABYASACHI BHOI',
  'image': None,
  'display_name': 'sabya',
  'pin_change_date': '24-01-2024 10:24:23',
  'email_id': 'sabyabhoi@gmail.com',
  'pwd_change_date': '24-10-2021 14:39:07;90',
  'PAN': '---------',
  'mobile_number': '8527854263',
  'totp': False,
  'pwd_to_expire': 90}}

# Data API


## Using SQLite3 Database


In [3]:
db = sql.connect("../res/realtime.db", check_same_thread=False)
c = db.cursor()


def create_table(token):
    c.execute(
        f"CREATE TABLE IF NOT EXISTS {token} (ts datetime primary key, price real(15, 5))"
    )

    try:
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()

## Live Datafeeds


In [4]:
def onmessage(message):
    """
    Callback function to handle incoming messages from the FyersDataSocket WebSocket.

    Parameters:
        message (dict): The received message from the WebSocket.

    """
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    query = f"INSERT INTO {s[0]} VALUES('{current_time}', {message['ltp']})"
    print(query)

    c.execute(query)

In [69]:
def onerror(message):
    """
    Callback function to handle WebSocket errors.

    Parameters:
        message (dict): The error message received from the WebSocket.


    """
    print("Error:", message)


def onclose(message):
    """
    Callback function to handle WebSocket connection close events.
    """
    print("Connection closed:", message)
    try:
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()


def onopen():
    """
    Callback function to subscribe to data type and symbols upon WebSocket connection.

    """
    # Specify the data type and symbols you want to subscribe to
    data_type = "SymbolUpdate"

    # Subscribe to the specified symbols and data type
    symbols = ["NSE:NIFTY50-INDEX"]
    fyers.subscribe(symbols=symbols, data_type=data_type)
    print(symbols)
    # Keep the socket running to receive real-time data
    fyers.keep_running()

In [70]:
# Create a FyersDataSocket instance with the provided parameters
fyers = data_ws.FyersDataSocket(
    access_token=access_token,  # Access token in the format "appid:accesstoken"
    log_path="",  # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=True,  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,  # Save response in a log file instead of printing it.
    reconnect=True,  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,  # Callback function to subscribe to data upon connection.
    on_close=onclose,  # Callback function to handle WebSocket connection close events.
    on_error=onerror,  # Callback function to handle WebSocket errors.
    on_message=onmessage,  # Callback function to handle incoming messages from the WebSocket.
)

# Establish a connection to the Fyers WebSocket
fyers.connect()

Error: KeyError: The key 'ltp' is missing in the response.
Error: KeyError: The key 'ltp' is missing in the response.
['NSE:NIFTY50-INDEX']


Error: KeyError: The key 'ltp' is missing in the response.
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:28.966', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:29.379', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:30.380', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:31.404', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:32.380', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:33.395', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:34.389', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:35.423', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:36.391', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:37.385', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:38.387', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:39.388', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:40.679', 2

In [71]:
fyers.close_connection()

## Historical data


In [9]:
response = fyers.history(
    data={
        "symbol": "NSE:SBIN-EQ",
        "resolution": "3",
        "date_format": "1",
        "range_from": "2024-01-05",
        "range_to": "2024-01-23",
    }
)
print(response)

{'candles': [[1704426300, 645, 647.45, 645, 646.25, 425786], [1704426480, 646.2, 647.75, 646.2, 646.8, 243890], [1704426660, 646.9, 650, 646.8, 649.2, 609582], [1704426840, 649.6, 649.6, 648, 648.45, 232613], [1704427020, 648.45, 649, 648, 648.5, 152385], [1704427200, 648.5, 649.3, 647.6, 647.65, 190065], [1704427380, 647.65, 648.15, 647.3, 647.8, 159607], [1704427560, 647.85, 648.35, 647.6, 648, 72267], [1704427740, 648, 648.2, 647.6, 648, 100279], [1704427920, 648, 648, 647.65, 648, 50960], [1704428100, 647.85, 648.1, 647.1, 647.4, 122488], [1704428280, 647.4, 647.8, 647.15, 647.5, 69150], [1704428460, 647.45, 648.25, 647.35, 648.1, 61576], [1704428640, 648.1, 649, 648, 648.95, 138258], [1704428820, 648.95, 650.05, 648.8, 649.6, 584093], [1704429000, 649.6, 650.5, 649, 649.1, 289064], [1704429180, 649.15, 650, 648.7, 649.9, 111521], [1704429360, 649.8, 650.15, 649.35, 649.75, 226046], [1704429540, 649.75, 649.85, 649.1, 649.5, 58578], [1704429720, 649.5, 650.2, 649.25, 650, 147560], 

In [10]:
df = pd.DataFrame(response["candles"])
df.columns = ["ts", "Open", "High", "Low", "Close", "Volume"]
df

,ts,Open,High,Low,Close,Volume
0,1704426300,645.00,647.45,645.00,646.25,425786
1,1704426480,646.20,647.75,646.20,646.80,243890
2,1704426660,646.90,650.00,646.80,649.20,609582
3,1704426840,649.60,649.60,648.00,648.45,232613
4,1704427020,648.45,649.00,648.00,648.50,152385
...,...,...,...,...,...,...
1620,1706003100,605.45,605.55,604.25,604.65,481332
1621,1706003280,604.70,605.90,604.30,605.05,561024
1622,1706003460,605.05,606.65,605.00,606.05,480432
1623,1706003640,606.35,606.65,605.65,605.95,444696


In [11]:
df.index = pd.to_datetime(df.ts, unit="s")
df = df.drop(columns=["ts"])
df

,Open,High,Low,Close,Volume
ts,,,,,
2024-01-05 03:45:00,645.00,647.45,645.00,646.25,425786
2024-01-05 03:48:00,646.20,647.75,646.20,646.80,243890
2024-01-05 03:51:00,646.90,650.00,646.80,649.20,609582
2024-01-05 03:54:00,649.60,649.60,648.00,648.45,232613
2024-01-05 03:57:00,648.45,649.00,648.00,648.50,152385
...,...,...,...,...,...
2024-01-23 09:45:00,605.45,605.55,604.25,604.65,481332
2024-01-23 09:48:00,604.70,605.90,604.30,605.05,561024
2024-01-23 09:51:00,605.05,606.65,605.00,606.05,480432


In [12]:
df.to_parquet("../res/sbin.parquet")